# Shark League Score
## Import Python libraries

In [1]:
import importlib
import sys

# Remove cached modules to force reload
if 'swc_simple_client' in sys.modules:
    del sys.modules['swc_simple_client']

import pandas as pd
import numpy as np
import logging
import swc_simple_client as swc


## Configure logging

In [2]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='shark_notebook.log', level=logging.INFO,)

## Setup notebook variables

In [3]:
base_url = "http://localhost:8000"


## Get Max Scores
### USe endpoint: LIST_WEEKS_ENDPOINT

In [4]:
week_api_response = swc.call_api_endpoint(base_url, swc.LIST_WEEKS_ENDPOINT)
print(f"Response status code: {week_api_response.status_code}")
print(f"Response content-type: {week_api_response.headers.get('content-type', 'N/A')}")

if week_api_response.status_code == 200:
    # Check if response is actually JSON
    content_type = week_api_response.headers.get('content-type', '')
    if 'application/json' in content_type:
        weeks_df = pd.DataFrame(week_api_response.json())
        weeks_df['year'] = weeks_df['week_number'].str.slice(0,4).astype(int)

        max_totals_grouped_df = weeks_df.groupby('year').agg(
            ppr_12_max_points=('ppr_12_max_points', 'sum'),
            half_ppr_8_max_points=('half_ppr_8_max_points', 'sum'))

        display(max_totals_grouped_df)
        logging.info("Max total points per scoring format per year calculated.")
    else:
        print(f"ERROR: Response is not JSON. Content-Type: {content_type}")
        print(f"Response text (first 500 chars): {week_api_response.text[:500]}")
else:
    print(f"API Error: {week_api_response.status_code}")
    print(f"Response text (first 500 chars): {week_api_response.text[:500]}")


Response status code: 500
Response content-type: N/A
API Error: 500
Response text (first 500 chars): Network error


## Get League Scoring Type
### Use Endpoint: LIST_LEAGUES_ENDPOINT

In [15]:
league_api_response = swc.call_api_endpoint(base_url, swc.LIST_LEAGUES_ENDPOINT)
print(f"Response status code: {league_api_response.status_code}")
print(f"Response content-type: {league_api_response.headers.get('content-type', 'N/A')}")

leagues_df = pd.DataFrame(league_api_response.json())
leagues_df = leagues_df.drop(columns=['teams', 'last_changed_date'])
display(leagues_df)

Response status code: 200
Response content-type: application/json


,league_id,league_name,scoring_type,league_size
0,5001,Pigskin Prodigal Fantasy League,PPR,12
1,5002,Recurring Champions League,Half-PPR,8
2,5003,AHAHFZZFFFL,Half-PPR,8
3,5004,Gridiron Gurus Fantasy League,PPR,12
4,5005,Best League Ever,PPR,12
